In [1]:
import pandas as pd

# 1. Cargar los datos
path = "C:/Users/USUARIO/Desktop/SOYhENRY/sistema de recomendacion/Movies/CSV/"

# Archivos CSV
belongs_to_collection = pd.read_csv(path + "belongs_to_collection_clean.csv")
movies_with_genres = pd.read_csv(path + "movies_with_genres.csv")
crew_desanidado = pd.read_csv(path + "crew_desanidado.csv")
all_genres = pd.read_csv(path + "all_genres.csv")



In [3]:
# 2. Explorar cada dataset
# Mostrar primeras filas y estructura
def explore_data(df, name):
    print(f"\nDataset: {name}")
    print(df.head())
    print(f"Shape: {df.shape}")
    print("\nNull values:")
    print(df.isnull().sum())
    print("\nBasic statistics:")
    print(df.describe(include='all'))


In [4]:
#Aplicar la función a cada archivo
explore_data(belongs_to_collection, "Belongs to Collection")



Dataset: Belongs to Collection
   movie_id                            name
0     10194            Toy Story Collection
1    119050       Grumpy Old Men Collection
2     96871  Father of the Bride Collection
3       645           James Bond Collection
4    117693                Balto Collection
Shape: (4491, 2)

Null values:
movie_id    0
name        0
dtype: int64

Basic statistics:
             movie_id             name
count     4491.000000             4491
unique            NaN             1695
top               NaN  The Bowery Boys
freq              NaN               29
mean    184108.678913              NaN
std     141605.338687              NaN
min         10.000000              NaN
25%      86027.000000              NaN
50%     141615.000000              NaN
75%     294172.000000              NaN
max     480160.000000              NaN


In [5]:
#Aplicar la función a cada archivo
explore_data(movies_with_genres, "Movies with Genres")


Dataset: Movies with Genres
  movie_id        genre_ids
0      862  [16, 35, 10751]
1     8844  [12, 14, 10751]
2    15602      [10749, 35]
3    31357  [35, 18, 10749]
4    11862             [35]
Shape: (45466, 2)

Null values:
movie_id     0
genre_ids    0
dtype: int64

Basic statistics:
       movie_id genre_ids
count     45466     45466
unique    45436      4069
top      141971      [18]
freq          3      5000


In [6]:
#Aplicar la función a cada archivo
explore_data(crew_desanidado, "Crew Desanidado")



Dataset: Crew Desanidado
   movie_id                 credit_id department  gender     id         job  \
0       862  52fe4284c3a36847f8024f49  Directing       2   7879    Director   
1       862  52fe4284c3a36847f8024f4f    Writing       2  12891  Screenplay   
2       862  52fe4284c3a36847f8024f55    Writing       2      7  Screenplay   
3       862  52fe4284c3a36847f8024f5b    Writing       2  12892  Screenplay   
4       862  52fe4284c3a36847f8024f61    Writing       0  12893  Screenplay   

             name  
0   John Lasseter  
1     Joss Whedon  
2  Andrew Stanton  
3      Joel Cohen  
4    Alec Sokolow  
Shape: (464314, 7)

Null values:
movie_id      0
credit_id     0
department    0
gender        0
id            0
job           0
name          0
dtype: int64

Basic statistics:
             movie_id                 credit_id  department         gender  \
count   464314.000000                    464314      464314  464314.000000   
unique            NaN                    46383

In [7]:
#Aplicar la función a cada archivo
explore_data(all_genres, "All Genres")



Dataset: All Genres
   genre_ids       name
0         16  Animation
1         35     Comedy
2      10751     Family
3         12  Adventure
4         14    Fantasy
Shape: (32, 2)

Null values:
genre_ids    0
name         0
dtype: int64

Basic statistics:
           genre_ids       name
count      32.000000         32
unique           NaN         32
top              NaN  Animation
freq             NaN          1
mean     8271.437500        NaN
std      9120.501401        NaN
min        12.000000        NaN
25%        36.750000        NaN
50%      7760.500000        NaN
75%     10871.500000        NaN
max     33751.000000        NaN


In [8]:
# 3. Insights preliminares
# Decodificar géneros en Movies_with_genres
movies_with_genres = movies_with_genres.explode("genre_ids")
movies_with_genres["genre_ids"] = movies_with_genres["genre_ids"].astype(str)
all_genres["genre_ids"] = all_genres["genre_ids"].astype(str)

genre_map = dict(zip(all_genres["genre_ids"], all_genres["name"]))
movies_with_genres["genre_name"] = movies_with_genres["genre_ids"].map(genre_map)

print("\nTop 5 movies with decoded genres:")
print(movies_with_genres.head())


Top 5 movies with decoded genres:
  movie_id        genre_ids genre_name
0      862  [16, 35, 10751]        NaN
1     8844  [12, 14, 10751]        NaN
2    15602      [10749, 35]        NaN
3    31357  [35, 18, 10749]        NaN
4    11862             [35]        NaN


In [9]:

# Roles clave en Crew
crew_roles = crew_desanidado.groupby("job").size().sort_values(ascending=False)
print("\nTop roles in crew:")
print(crew_roles.head(10))



Top roles in crew:
job
Director                   49048
Producer                   43555
Writer                     30431
Screenplay                 25163
Editor                     23800
Director of Photography    20659
Executive Producer         18866
Original Music Composer    15790
Casting                    12635
Art Direction              11689
dtype: int64


In [12]:
# Asegurarse de que 'movie_id' sea del mismo tipo en ambos DataFrames
belongs_to_collection["movie_id"] = belongs_to_collection["movie_id"].astype(str)
movies_with_genres["movie_id"] = movies_with_genres["movie_id"].astype(str)

# Realizar el merge
collection_genres = pd.merge(belongs_to_collection, movies_with_genres, on="movie_id", how="inner")

# Agrupar y resumir los géneros por colección
collection_genres_summary = (
    collection_genres.groupby("name")["genre_name"]
    .apply(lambda x: x.value_counts().head(3))
)

print("\nTop genres per collection:")
print(collection_genres_summary)



Top genres per collection:
Series([], Name: genre_name, dtype: object)


In [13]:
# 4. Guardar resultados preliminares para análisis posterior
movies_with_genres.to_csv(path + "movies_with_decoded_genres.csv", index=False)
crew_roles.to_csv(path + "crew_roles_summary.csv")
collection_genres_summary.to_csv(path + "collection_genres_summary.csv")


In [14]:
genre_frequency = collection_genres["genre_name"].value_counts()
print("Frecuencia de géneros por colección:")
print(genre_frequency)


Frecuencia de géneros por colección:
Series([], Name: count, dtype: int64)


In [15]:
total_genre_distribution = all_genres["name"].value_counts()
print("Distribución total de géneros:")
print(total_genre_distribution)


Distribución total de géneros:
name
Animation                                1
Comedy                                   1
Family                                   1
Adventure                                1
Fantasy                                  1
Romance                                  1
Drama                                    1
Action                                   1
Crime                                    1
Thriller                                 1
Horror                                   1
History                                  1
Science Fiction                          1
Mystery                                  1
War                                      1
Foreign                                  1
Music                                    1
Documentary                              1
Western                                  1
TV Movie                                 1
Carousel Productions                     1
Vision View Entertainment                1
Telescene Film Gro

In [16]:
null_summary = collection_genres.isnull().sum()
print("\nResumen de valores nulos:")
print(null_summary)



Resumen de valores nulos:
movie_id      0
name          0
genre_ids     0
genre_name    0
dtype: int64


In [17]:
genre_roles = pd.merge(movies_with_genres, crew_desanidado, on="movie_id", how="inner")
genre_roles_summary = genre_roles.groupby(["genre_name", "job"])["movie_id"].count()
print("Roles más comunes por género:")
print(genre_roles_summary.sort_values(ascending=False).head(10))


ValueError: You are trying to merge on object and int64 columns for key 'movie_id'. If you wish to proceed you should use pd.concat

In [18]:
sample_movies = collection_genres["movie_id"].unique()
sample_data = movies_with_genres[movies_with_genres["movie_id"].isin(sample_movies)]
print("Tamaño del subconjunto de datos:", sample_data.shape)


Tamaño del subconjunto de datos: (0, 3)


In [19]:
print(movies_with_genres.dtypes)
print(crew_desanidado.dtypes)


movie_id      object
genre_ids     object
genre_name    object
dtype: object
movie_id       int64
credit_id     object
department    object
gender         int64
id             int64
job           object
name          object
dtype: object


In [20]:
movies_with_genres["movie_id"] = movies_with_genres["movie_id"].astype(str)
crew_desanidado["movie_id"] = crew_desanidado["movie_id"].astype(str)


In [21]:
genre_roles = pd.merge(movies_with_genres, crew_desanidado, on="movie_id", how="inner")


In [22]:
genre_roles_summary = genre_roles.groupby(["genre_name", "job"])["movie_id"].count()
print("Roles más comunes por género:")
print(genre_roles_summary.sort_values(ascending=False).head(10))


Roles más comunes por género:
Series([], Name: movie_id, dtype: int64)


In [23]:
genre_roles["movie_id"] = genre_roles["movie_id"].astype(int)
